In [16]:
from myAE import SDC_AE
import torch

manager = SDC_AE()
manager.prepare_data()
manager.prepare_model(22, 12)

def calc_loss(model, x, F, device):

  oh = F.one_hot(torch.arange(10, device=device))
  ps = torch.cat([torch.zeros(10, 2, device=device), oh], dim=1) * 10
  
  h = model.get_submodule("encoder")(x)
  z = h[:, :12]
  log_prob = h[:, 12:]
  max_prob = log_prob.argmax(dim=1)
  one_hot_prob = F.one_hot(max_prob, num_classes=10)
  dist = (((torch.stack([z]*10, dim=1) - ps).square().sum(dim=2) * one_hot_prob).max(dim=1).values).mean()
  pred = model.get_submodule("decoder")(z)
  
  ll = F.binary_cross_entropy(pred, x, reduction='sum')/len(x)
  loss = ll + dist
  return loss

hist = manager.train(calc_loss, epochs=50)

Now, it is working on cuda:1.


100%|██████████| 50/50 [04:31<00:00,  5.43s/it]


In [17]:
manager.save_current_model('dc_ae_12dim_10s_50ep')

In [20]:
with torch.no_grad():
  manager.model.to('cpu')
  manager.model.eval()
  for x, y in manager.train_dataloader:
    x = x.reshape([-1, 784])
    h = manager.model.get_submodule('encoder')(x)
    log_prob = h[:, 12:]
    max_prob = log_prob.argmax(dim=1)
    for idx in range(10):
      print(max_prob[y==idx])
    break

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
